# Process ACCESS-ESM1-5 to NEM region

In [1]:
from dask.distributed import Client,LocalCluster
from dask_jobqueue import PBSCluster

In [2]:
# One node on Gadi has 48 cores - try and use up a full node before going to multiple nodes (jobs)

walltime = '00:30:00'
cores = 24
memory = str(4 * cores) + 'GB'

cluster = PBSCluster(walltime=str(walltime), cores=cores, memory=str(memory), processes=cores,
                     job_extra_directives=['-q normal',
                                           '-P w42',
                                           '-l ncpus='+str(cores),
                                           '-l mem='+str(memory),
                                           '-l storage=gdata/w42+gdata/rt52+gdata/fs38'],
                     local_directory='$TMPDIR',
                     job_directives_skip=["select"])
                     # python=os.environ["DASK_PYTHON"])

/g/data/w42/dr6273/apps/conda/envs/pangeo/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 36987 instead
  warnings.warn(


In [3]:
cluster.scale(jobs=1)
client = Client(cluster)

In [4]:
client#.restart()

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: /proxy/36987/status,
Dashboard: /proxy/36987/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://10.6.121.2:41381,Workers: 0
Dashboard: /proxy/36987/status,Total threads: 0
Started: Just now,Total memory: 0 B


In [5]:
import xarray as xr

import os

In [6]:
path_hist = "/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical"
path_proj = "/g/data/fs38/publications/CMIP6/ScenarioMIP/CSIRO/ACCESS-ESM1-5/"

In [7]:
def NEM_process(variable, time_str, preprocess):
    """
    Concat historical runs from 1950, select NEM region.
    
    variable: str, variable name
    time_str: str, name of time interval e.g. day
    preprocess: function to pass to preprocess in open_mfdataset
    """
    base_path = "/g/data/fs38/publications/CMIP6/CMIP/CSIRO/ACCESS-ESM1-5/historical"
    members = os.listdir(base_path)
    
    ds_list = []
    for member in members:
        path = base_path + "/" + member + "/" + time_str + "/" + variable + "/gn/latest/" + variable + "_" + time_str + "_ACCESS-ESM1-5_historical_" + member

        ds = xr.open_mfdataset([
            path + "_gn_19500101-19991231.nc",
            path + "_gn_20000101-20141231.nc"
        ],
            # parallel=True,
            join="exact",
            preprocess=preprocess,
            engine="netcdf4",
            compat="override",
            coords="minimal",
        )
        m = int(member.split("i")[0][1:])
        ds = ds.expand_dims({"member": [m]})
        ds_list.append(ds)
    return xr.concat(ds_list, dim="member").sortby("member")

### Temperature (2m)

In [8]:
def preprocess_tas(ds):
    ds = ds["tas"].to_dataset(name="tas")
    ds = ds.drop("height")
    return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [14]:
tas = NEM_process("tas", "day", preprocess_tas)

In [18]:
tas = tas.chunk({"member": 5})

In [22]:
tas.to_zarr(
    "/g/data/w42/dr6273/work/data/CMIP6/ACCESS-ESM1-5/historical/day/tas/tas_day_ACCESS-ESM1-5_historical_1950-2014_NEM.zarr",
    mode="w",
    consolidated=True
)

### Daily max temperature (2m)

In [39]:
def preprocess_tasmax(ds):
    ds = ds["tasmax"].to_dataset(name="tasmax")
    ds = ds.drop("height")
    return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [40]:
tasmax = NEM_process("tasmax", "day", preprocess_tasmax)

In [41]:
tasmax = tasmax.chunk({"member": 5})

In [42]:
tasmax.to_zarr(
    "/g/data/w42/dr6273/work/data/CMIP6/ACCESS-ESM1-5/historical/day/tasmax/tasmax_day_ACCESS-ESM1-5_historical_1950-2014_NEM.zarr",
    mode="w",
    consolidated=True
)

### Shortwave radiation at surface

In [32]:
def preprocess_rsds(ds):
    ds = ds["rsds"].to_dataset(name="rsds")
    return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [35]:
rsds = NEM_process("rsds", "day", preprocess_rsds)

In [36]:
rsds = rsds.chunk({"member": 5})

In [38]:
rsds.to_zarr(
    "/g/data/w42/dr6273/work/data/CMIP6/ACCESS-ESM1-5/historical/day/rsds/rsds_day_ACCESS-ESM1-5_historical_1950-2014_NEM.zarr",
    mode="w",
    consolidated=True
)

### Surface wind speed (~10 m)

In [44]:
def preprocess_sfcWind(ds):
    ds = ds["sfcWind"].to_dataset(name="sfcWind")
    return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [45]:
sfcWind = NEM_process("sfcWind", "day", preprocess_sfcWind)

In [36]:
sfcWind = rssfcWindds.chunk({"member": 5})

In [47]:
sfcWind.to_zarr(
    "/g/data/w42/dr6273/work/data/CMIP6/ACCESS-ESM1-5/historical/day/sfcWind/sfcWind_day_ACCESS-ESM1-5_historical_1950-2014_NEM.zarr",
    mode="w",
    consolidated=True
)

### Precip

In [11]:
def preprocess_pr(ds):
    ds = ds["pr"].to_dataset(name="pr")
    return ds.sel(lon=slice(133, 155), lat=slice(-45, -10))

In [12]:
pr = NEM_process("pr", "day", preprocess_pr)

In [14]:
pr = pr.chunk({"member": 5})

In [15]:
pr.to_zarr(
    "/g/data/w42/dr6273/work/data/CMIP6/ACCESS-ESM1-5/historical/day/pr/pr_day_ACCESS-ESM1-5_historical_1950-2014_NEM.zarr",
    mode="w",
    consolidated=True
)

# Close cluster

In [16]:
client.close()
cluster.close()